In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn.apionly as sns

%matplotlib inline


/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = other[key]
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  self[key] = other[key]
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 


In [29]:
df = pd.read_csv('data/cell2celltrain.csv')
df['Churn'].describe()

51047 - 36336




14711

In [33]:
df = pd.read_csv('data/cell2cellholdout.csv')
df.describe()

df[df['Churn'] == 'Yes']['Churn'].describe()


/usr/local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Churn, dtype: float64

In [1]:



bins=10

#Stack the data
plt.figure()
plt.hist(df[df['Churn'] == 'Yes']['MonthsInService'], bins, label='Yes', stacked=True, alpha=0.4)
plt.hist(df[df['Churn'] == 'No']['MonthsInService'], bins, label='No', stacked=True,  alpha=0.4)
plt.legend()
plt.show()




NameError: name 'plt' is not defined

In [25]:

df['CurrentEquipmentDays'].describe()


count    51046.000000
mean       380.545841
std        253.801982
min         -5.000000
25%        205.000000
50%        329.000000
75%        515.000000
max       1812.000000
Name: CurrentEquipmentDays, dtype: float64